#  <span style="color:green">  -------- FAKE JOB POSTINGS -------- </span>


![Sample Image](https://akpsi.org/wp-content/uploads/2020/12/AdobeStock_134022431-scaled.jpeg)


In [1]:
import pandas as pd
from keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
job = pd.read_csv(r"fake_job_postings.csv")
job

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


In [3]:
job.shape

(17880, 18)

In [4]:
job.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [5]:
#Creating dataframe to take only those columns which we want
df = pd.DataFrame()
df['X'] = job['company_profile'].apply(str)+ ' '+job['description'].apply(str)+' '+job['requirements'].apply(str)+' '+job['required_experience'].apply(str)+' '+job['has_company_logo'].apply(str)+' '+job['industry'].apply(str)+' '+job['benefits'].apply(str)+' '+job['required_education'].apply(str)

In [6]:
df['Y'] = job.fraudulent

In [7]:
df.head()

,X,Y
0,"We're Food52, and we've created a groundbreaki...",0
1,"90 Seconds, the worlds Cloud Video Production ...",0
2,Valor Services provides Workforce Solutions th...,0
3,Our passion for improving quality of life thro...,0
4,SpotSource Solutions LLC is a Global Human Cap...,0


In [8]:
#Converting into Lower
df.X = df.X.str.lower()
df.head()

,X,Y
0,"we're food52, and we've created a groundbreaki...",0
1,"90 seconds, the worlds cloud video production ...",0
2,valor services provides workforce solutions th...,0
3,our passion for improving quality of life thro...,0
4,spotsource solutions llc is a global human cap...,0


In [9]:
df.Y.unique()

array([0, 1], dtype=int64)

In [10]:
df_x = df.iloc[:,0]
df_y = df.iloc[:,1]

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_x , test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.2)

In [13]:
train_y = to_categorical(train_y)

In [14]:
df.shape

(17880, 2)

In [15]:
max_num_words = 10000      #select 10000 unique words from entire words
seq_len = 50              #out of 10000 w selct only 5o words from each document 
embedding_size = 100       #for each word we will create emedding of size 100

In [16]:
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, LSTM

In [17]:
tokenizer = Tokenizer(num_words=max_num_words) #assign a unique number

tokenizer.fit_on_texts(df.X)                   #but on your entire data not on train , test
 
train_x = tokenizer.texts_to_sequences(train_x)#will convert the text to sequences os IDs

In [18]:
train_x = pad_sequences(train_x, maxlen = seq_len)

In [19]:
from keras.layers import Embedding

In [20]:
test_x = tokenizer.texts_to_sequences(test_x)
test_x = pad_sequences(test_x, maxlen = seq_len)

model = Sequential()
model.add(Embedding(input_dim = max_num_words,
                   input_length = seq_len,
                   output_dim = embedding_size))

model.add(LSTM(5))
model.add(Dense(2, activation='softmax'))

from tensorflow.keras.optimizers import Adam
adam = Adam(lr = .1)

model.compile(optimizer= adam, loss='categorical_crossentropy', metrics = ['accuracy'])

In [21]:
model.fit(train_x, train_y, epochs= 5, batch_size = 32, validation_split = .2) 

Epoch 1/5
358/358 [==============================] - 19s 43ms/step - loss: 0.2209 - accuracy: 0.9530 - val_loss: 0.1264 - val_accuracy: 0.9602
Epoch 2/5
358/358 [==============================] - 14s 40ms/step - loss: 0.0861 - accuracy: 0.9751 - val_loss: 0.1035 - val_accuracy: 0.9724
Epoch 3/5
358/358 [==============================] - 14s 39ms/step - loss: 0.0521 - accuracy: 0.9864 - val_loss: 0.1030 - val_accuracy: 0.9647
Epoch 4/5
358/358 [==============================] - 14s 39ms/step - loss: 0.0335 - accuracy: 0.9926 - val_loss: 0.1131 - val_accuracy: 0.9762
Epoch 5/5
358/358 [==============================] - 14s 39ms/step - loss: 0.0245 - accuracy: 0.9944 - val_loss: 0.1163 - val_accuracy: 0.9661


In [22]:
from sklearn.metrics import classification_report, confusion_matrix

pred = model.predict(test_x)
pred_classes = pred.argmax(axis = 1)
print(classification_report(test_y, pred_classes))

112/112 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3404
           1       0.59      0.62      0.60       172

    accuracy                           0.96      3576
   macro avg       0.78      0.80      0.79      3576
weighted avg       0.96      0.96      0.96      3576



In [34]:
# Function to check if a job posting is fake or not
def check_fake_job_posting(text):
    text = text.lower()
    sequences = tokenizer.texts_to_sequences([text])
    sequences = pad_sequences(sequences, maxlen=seq_len)
    pred = model.predict(sequences)
    if pred.argmax(axis=1)[0] == 0:
        return "This job posting is true"
    else:
        return "This job posting is fake"

# Sample text to check if fake or not
X = "Karry to go to France in gesture of sympathy"
print(check_fake_job_posting(X))

1/1 [==============================] - 0s 27ms/step
This job posting is true
